In [18]:
import os
import zipfile
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image

In [12]:
dataset_zip = 'enfermedades_piel.v1i.tensorflow.zip'
extract_dir = 'data/'

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [16]:
train_csv_path = os.path.join(extract_dir, 'train', '_annotations.csv')
valid_csv_path = os.path.join(extract_dir, 'valid', '_annotations.csv')
test_csv_path = os.path.join(extract_dir, 'test', '_annotations.csv')

In [14]:
def load_data(csv_path, base_dir):
    data = pd.read_csv(csv_path)
    images = []
    labels = []
    for index, row in data.iterrows():
        img_path = os.path.join(base_dir, row['filename'])
        image = Image.open(img_path)
        image = image.resize((img_width, img_height))
        image = tf.keras.preprocessing.image.img_to_array(image)
        images.append(image)
        labels.append(row['class'])
    return np.array(images), np.array(labels)

In [3]:
batch_size = 32
img_height = 150
img_width = 150
epochs = 20

In [19]:
train_images, train_labels = load_data(train_csv_path, os.path.join(extract_dir, 'train'))
valid_images, valid_labels = load_data(valid_csv_path, os.path.join(extract_dir, 'valid'))
test_images, test_labels = load_data(test_csv_path, os.path.join(extract_dir, 'test'))

In [20]:
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
valid_labels_encoded = label_encoder.transform(valid_labels)
test_labels_encoded = label_encoder.transform(test_labels)

In [21]:
num_classes = len(label_encoder.classes_)
train_labels_categorical = to_categorical(train_labels_encoded, num_classes)
valid_labels_categorical = to_categorical(valid_labels_encoded, num_classes)
test_labels_categorical = to_categorical(test_labels_encoded, num_classes)

In [22]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_images, train_labels_categorical, batch_size=batch_size)
validation_generator = validation_datagen.flow(valid_images, valid_labels_categorical, batch_size=batch_size)


In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.summary()

c:\Users\Lenovo\Pemrograman\Bangkit\cap-env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,454,660 (13.18 MB)

 Trainable params: 3,454,660 (13.18 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('val_accuracy') > 0.90:
            self.model.stop_training = True

callbacks = myCallback()

In [26]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_images) // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(valid_images) // batch_size,
    callbacks=[callbacks]
)

Epoch 1/20


c:\Users\Lenovo\Pemrograman\Bangkit\cap-env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


110/110 ━━━━━━━━━━━━━━━━━━━━ 57s 491ms/step - accuracy: 0.3177 - loss: 1.3621 - val_accuracy: 0.5719 - val_loss: 1.0512
Epoch 2/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5625 - loss: 1.0845 - val_accuracy: 0.5000 - val_loss: 0.8921


C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\contextlib.py:135: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(type, value, traceback)


Epoch 3/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 54s 481ms/step - accuracy: 0.5100 - loss: 1.0912 - val_accuracy: 0.6531 - val_loss: 0.8159
Epoch 4/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7188 - loss: 0.6992 - val_accuracy: 0.6333 - val_loss: 0.7897
Epoch 5/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 51s 451ms/step - accuracy: 0.6323 - loss: 0.8801 - val_accuracy: 0.7250 - val_loss: 0.7429
Epoch 6/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7188 - loss: 0.6115 - val_accuracy: 0.7000 - val_loss: 0.7094
Epoch 7/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 48s 432ms/step - accuracy: 0.6881 - loss: 0.7972 - val_accuracy: 0.6906 - val_loss: 0.6690
Epoch 8/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6250 - loss: 0.8871 - val_accuracy: 0.7667 - val_loss: 0.6880
Epoch 9/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 43s 382ms/step - accuracy: 0.6937 - loss: 0.7526 - val_accuracy: 0.6719 - val_loss: 0.6998
Epoch 10/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7500 - loss: 0.6363 - val

In [27]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow(test_images, test_labels_categorical, batch_size=batch_size)
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_images) // batch_size)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - accuracy: 0.6661 - loss: 0.7892
Test accuracy: 66.25%


In [28]:
model.save('skin_disease_classifier.h5')